In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool

In [11]:
url = 'https://dblp.org/pid/125/1647.html' #
#url = 'https://dblp.org/pers/a/A:Ambha.html' #None
url = 'https://dblp.org/pid/73/1296.html' #MIT

In [12]:
r = requests.get(url=url)
soup = BeautifulSoup(r.content, 'html.parser')

In [13]:
##en la seccion info
soup.find("div", {"id": "info-section"}).find("span").text

'Massachusetts Institute of Technology, Cambridge, USA'

In [14]:
def extract_affiliation(url):
    r = requests.get(url=url)
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        return soup.find("div", {"id": "info-section"}).find("span").text
    except Exception as e:
        return str(e)
    

In [ ]:
def extract_affiliation_df(url):
    r = requests.get(url=url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        aff = soup.find("div", {"id": "info-section"}).find("span").text
        return soup.find("div", {"id": "info-section"}).find("span").text
    except Exception as e:
        return str(e)

In [15]:
def make_pool(func, params, threads=None):
    pool = Pool(threads)
    data = pool.map(func, params)
    pool.close()
    pool.join()
    del pool
    return [x for x in data]

In [16]:
data_path = 'dfs_posicion_inicial/'
data_path = 'dfs_pos_i_20210727/'

In [17]:
l = [pd.read_csv(os.path.join(data_path,x)) for x in os.listdir(data_path)]

In [18]:
df = pd.concat(l)

In [19]:
sample = df.sample(100)

In [56]:
sample['aff'] = sample.ref.map(extract_affiliation)

In [20]:
sample['aff2'] = make_pool(extract_affiliation, sample.ref)

In [21]:
sample['aff2'].value_counts()

'NoneType' object has no attribute 'text'                                                                      98
La Trobe University, Department of Computer Science and Computer Engineering, Melbourne, Australia              1
University of Lisbon, Instituto Superior Técnico, Centro de Recursos Naturais e Ambiente (CERENA), Portugal     1
Name: aff2, dtype: int64

In [64]:
sample['aff'].value_counts()

'NoneType' object has no attribute 'text'                                                                   99
Delft University of Technology, Delft Center of Systems and Control, Delft, The Netherlands (and 3 more)     1
Name: aff, dtype: int64

In [22]:
sample[sample.aff=='0002']

AttributeError: 'DataFrame' object has no attribute 'aff'

In [ ]:
df['aff'] = make_pool(extract_affiliation, df.ref)

In [ ]:
df.to_pickle('df_20210729_affiliation.pkl')